### Key Points 

In [1]:
!pip install mediapipe==0.10.9 opencv-python matplotlib scikit-learn tensorflow==2.13.0rc2

### Importing Dependencies 

In [2]:
import cv2 
from matplotlib import pyplot as plt
import numpy as np 
import os 
import time 
import mediapipe as mp 

### Extracting Key Points

In [3]:
mp_holistic = mp.solutions.holistic                       # Holistic model 
mp_drawing = mp.solutions.drawing_utils                  # Drawing utilities 

In [13]:
def mediapipe_detection(img, model): 
    """
        Processes an image using a MediaPipe model to detect holistic features.

        Parameters:
        - image: The input image in BGR format (as read by OpenCV).
        - model: A MediaPipe model instance configured for holistic detection (e.g., mp_holistic.Holistic).

        Returns:
        - image_rgb: The input image converted back from RGB to BGR format after processing.
        - results: The detection results from the MediaPipe model, including landmarks for face, pose, and hands.
    """
    image_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)              # COLOR CONVERSION BGR 2 RGB
    results = model.process(image_rgb.copy())                       # Make prediction on a copy of the image
    image_rgb = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)          # COLOR CONVERSION RGB 2 BGR 
    return image_rgb, results


def annotate_with_landmarks(img, res): 
    """
        Draws landmarks and connections for face, pose, and hands on an image.

        Parameters:
        - img: The input image where landmarks will be drawn, expected in BGR format.
        - res: The detection results containing landmarks detected by MediaPipe.
               It includes face_landmarks, pose_landmarks, left_hand_landmarks, and right_hand_landmarks.

        Returns:
        - None. The function directly modifies the input image to draw landmarks and connections.
        - These landmarks and connections are styled.
    """
    # Face Landmarks
    mp_drawing.draw_landmarks( 
        img, res.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
        mp_drawing.DrawingSpec(color=(247, 198, 246), thickness=1, circle_radius=1),  
        mp_drawing.DrawingSpec(color=(115, 61, 191), thickness=1, circle_radius=1))        
    # Pose connections
    mp_drawing.draw_landmarks( 
        img, res.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
        mp_drawing.DrawingSpec(color=(250, 249, 187), thickness=2, circle_radius=4),  
        mp_drawing.DrawingSpec(color=(158, 207, 255), thickness=2, circle_radius=2))           
    # right hand connections 
    mp_drawing.draw_landmarks( 
        img, res.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
        mp_drawing.DrawingSpec(color=(224, 224, 164), thickness=2, circle_radius=4),  
        mp_drawing.DrawingSpec(color=(167, 204, 169), thickness=2, circle_radius=2))      
    # left hand connections 
    mp_drawing.draw_landmarks( 
        img, res.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
        mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),  
        mp_drawing.DrawingSpec(color=(151, 199, 154), thickness=2, circle_radius=2))       



    

In [14]:
webcam = cv2.VideoCapture(0)                  # device number 0
with mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as holistic:
    while webcam.isOpened():                      # while the webcam is  turned on
        data = webcam.read()                      # get the data
        if not data[0]:
            break
        # make landmarks detection
        img, res = mediapipe_detection(data[1], holistic)
        # annoatate on video
        annotate_with_landmarks(img, res)   
        
        cv2.imshow('Camera Feed', img)        # display the image feed

        if cv2.waitKey(10) & 0xFF == ord('q'):    # close the feed using key "q"
            break

    # once the webcam is closed, close the display window
    webcam.release()                              
    cv2.destroyAllWindows()
    cv2.waitKey(1)  
    time.sleep(1)   

I0000 00:00:1709621188.208434       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1


### Getting Essential Key Point Values 

In [ ]:
curr_pose = []
for pose in res.pose_landmarks.landmark:
    pt = np.array([r.x, r.y, r.z, r.visibility])
    curr